# Autoamted Email Generator System
## For Job Application

### Call API for LLM model using groqcloud 

In [ ]:
from langchain_groq import ChatGroq

# init LLM 
llm = ChatGroq(
    temperature=0, 
    groq_api_key='input_API_key',  #groqcloud
    model_name="llama-3.3-70b-versatile"
)

### Job Web Scrapping 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
# web scrapping from career webpage
job_url = "input_url"
loader = WebBaseLoader(job_url)
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Mistral AI - Applied AI Engineer - Paris (Internship)Applied AI Engineer - Paris (Internship)ParisSolutions /On-siteApply for this jobAs an Applied Engineering Intern, you will work closely with our Applied AI Engineering team to facilitate the adoption of Mistral AI products among customers and collaborate with them to address complex technical challenges. This role is based in Paris, with an internship duration of 3 to 6 months. We are open to CIFRE programs as a continuation after the internship.Key ResponsibilitiesContribute to the deployment of state-of-the-art GenAI applications, driving technological transformation with our customers.Collaborate with researchers, AI engineers, and product engineers on complex customer projects.Work with the product and science team to continuously improve our product and model capabilities based on customer feedback.You may be a good fit ifYou are currently pursuing a degree in AI, data science, or a related field from a tier 1 engineering schoo

### Prompt LLM to get relevant information

In [3]:
from langchain_core.prompts import PromptTemplate

# init prompt to get relevant info
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
# getting prompt then pass it to model 
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content) #this is str

```json
{
  "role": "Applied AI Engineer",
  "experience": "Internship",
  "skills": [
    "Python",
    "Machine learning algorithms",
    "Natural language processing techniques",
    "MLOps",
    "Deep learning frameworks (e.g., PyTorch)",
    "Version control systems (e.g., Git)",
    "Linux shell environment",
    "HPC Environments"
  ],
  "description": "Contribute to the deployment of state-of-the-art GenAI applications, driving technological transformation with customers. Collaborate with researchers, AI engineers, and product engineers on complex customer projects. Work with the product and science team to continuously improve product and model capabilities based on customer feedback."
}
```


### Process result

In [4]:
from langchain_core.output_parsers import JsonOutputParser
# convert text to json
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Applied AI Engineer',
 'experience': 'Internship',
 'skills': ['Python',
  'Machine learning algorithms',
  'Natural language processing techniques',
  'MLOps',
  'Deep learning frameworks (e.g., PyTorch)',
  'Version control systems (e.g., Git)',
  'Linux shell environment',
  'HPC Environments'],
 'description': 'Contribute to the deployment of state-of-the-art GenAI applications, driving technological transformation with customers. Collaborate with researchers, AI engineers, and product engineers on complex customer projects. Work with the product and science team to continuously improve product and model capabilities based on customer feedback.'}

In [5]:
import pandas as pd

df = pd.read_csv("portfolio.csv")
df

,Techstack,Links
0,"Vue.js, Laravel, PostgreSQL",https://richardcheam.github.io/1Ainternship
1,"Python, PySpark, Excel, draw.io",https://richardcheam.github.io/2Ainternship
2,"R, survival, ggplot",https://richardcheam.github.io/project/HIV-sur...
3,"Machine Learning, Python, PyTorch",https://richardcheam.github.io/project/deep-le...
4,"Computer Vision, YOLO, OpenCV",https://richardcheam.github.io/project/footbal...
5,"LLM, RAG, VertexAI, GCP, Docker",https://richardcheam.github.io/project/llm


### Vector database

In [6]:
import uuid
import chromadb

# PersistentClient will create db on disk physically 
student = chromadb.PersistentClient('vectorstore')
collection = student.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [7]:
job = json_res
# get all portfolio links that match the job skills for email generator
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

### Prompt LLM to write mail

In [8]:
# prompt LLM to write email
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are a student who is pursuing a double degree, Engineer's degree and Master's degree, in Data Science at Paris-Saclay University.
        Throughout your studies, you have acquired different skills and have done numerous projects related to machine learning, deep learning, and so on.
        Right now, you are looking for an internship. 
        So, your job is to write a email to the recruiter as an application regarding the internship mentioned above describing your capability in fulfilling their needs or roles.
        Also add the most relevant ones from the following links to showcase your portfolio: {link_list}
        Remember you are a student applying for internship. 
        Do not provide a preamble.
        Also, your name is Bob
        ### EMAIL (NO PREAMBLE):

        """
        )

# pass prompt to llm
chain_email = prompt_email | llm

# pass {job} and {links} vars to mail
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Applied AI Engineer Internship

Dear Hiring Manager,

I am excited to apply for the Applied AI Engineer internship at your esteemed organization. As a student pursuing a double degree in Data Science at Paris-Saclay University, I am confident that my skills and experience make me an ideal candidate for this role.

Throughout my studies, I have acquired a strong foundation in machine learning, deep learning, and natural language processing techniques. I have worked on numerous projects that demonstrate my capabilities in these areas. For instance, my project on deep learning (https://richardcheam.github.io/project/deep-learning) showcases my understanding of deep learning frameworks and my ability to apply them to real-world problems. Additionally, my project on large language models (https://richardcheam.github.io/project/llm) highlights my knowledge of natural language processing techniques and my ability to work with complex models.

I am also proficient in P